In [1]:
import pandas as pd
import os
import torch

from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import (
    WidePreprocessor,
    TabPreprocessor,
    TextPreprocessor,
    ImagePreprocessor,
)
from pytorch_widedeep.models import (
    Wide,
    TabMlp,
    Vision,
    BasicRNN,
    WideDeep,
)
from pytorch_widedeep.losses import RMSELoss
from pytorch_widedeep.initializers import *
from pytorch_widedeep.callbacks import *

2022-04-04 13:56:33.827673: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-04 13:56:33.827700: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv("./RatingswithUI.csv")

In [3]:
ind = np.where([not isinstance(f, float) for f in df.feature])[0]
df = df.iloc[ind, :]

In [4]:
crossed_cols = [("verified", "tot_review_u>10")]
# add binary variable for tot_review
wide_cols = ["verified", "tot_review_u>10"]

cat_embed_cols = [("main_cat", 29)]

continuous_cols = ["avg_score_u", "avg_sentiment_u", 
                   "weighted_score_u", "avg_score", 
                   "avg_sentiment", "weighted_score",
                  "tot_review"]

text_col = "feature"
img_col = "img_col"

word_vectors_path = "./glove.6B/glove.6B.100d.txt"
img_path = "./data/Images for use"

target_col = "overall"


In [5]:
target = df[target_col].values

In [6]:
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = wide_preprocessor.fit_transform(df)

In [7]:
tab_preprocessor = TabPreprocessor(
    cat_embed_cols=cat_embed_cols,
    continuous_cols=continuous_cols,
)
X_tab = tab_preprocessor.fit_transform(df)

In [8]:
text_preprocessor = TextPreprocessor(
    word_vectors_path=word_vectors_path, text_col=text_col
)
X_text = text_preprocessor.fit_transform(df)

The vocabulary contains 1966 tokens
Indexing word vectors...
Loaded 400000 word vectors
Preparing embeddings matrix...
1761 words in the vocabulary had ./glove.6B/glove.6B.100d.txt vectors and appear more than 5 times


In [9]:
image_processor = ImagePreprocessor(img_col=img_col, img_path=img_path)
X_images = image_processor.fit_transform(df)

Reading Images from ./data/Images for use


  2%|▉                                          | 442/20000 [00:00<00:04, 4413.94it/s]

Resizing


100%|█████████████████████████████████████████| 20000/20000 [00:07<00:00, 2564.69it/s]


Computing normalisation metrics


In [10]:
wide = Wide(input_dim=np.unique(X_wide).shape[0], pred_dim=1)

# DeepDense: 2 Dense layers
tab_mlp = TabMlp(
    column_idx=tab_preprocessor.column_idx,
    cat_embed_input=tab_preprocessor.cat_embed_input,
    cat_embed_dropout=0.1,
    continuous_cols=continuous_cols,
    mlp_hidden_dims=[128, 64],
    mlp_dropout=0.1,
)

# DeepText: a stack of 2 LSTMs
basic_rnn = BasicRNN(
    vocab_size=len(text_preprocessor.vocab.itos),
    embed_matrix=text_preprocessor.embedding_matrix,
    n_layers=4,
    hidden_dim=64,
    rnn_dropout=0.5,
)

# Pretrained Resnet 18
resnet = Vision(pretrained_model_name="resnet34", n_trainable=4)

In [11]:
model = WideDeep(
    wide=wide,
    deeptabular=tab_mlp,
    deeptext=basic_rnn,
    deepimage=resnet,
    head_hidden_dims=[256, 128],
)

In [12]:
deep_params = []
for childname, child in model.named_children():
    if childname == "deeptabular":
        for n, p in child.named_parameters():
            if "embed_layer" in n:
                deep_params.append({"params": p, "lr": 1e-4})
            else:
                deep_params.append({"params": p, "lr": 1e-3})

In [13]:
wide_opt = torch.optim.Adam(model.wide.parameters(), lr=0.03)
deep_opt = torch.optim.Adam(deep_params)
text_opt = torch.optim.AdamW(model.deeptext.parameters())
img_opt = torch.optim.AdamW(model.deepimage.parameters())
head_opt = torch.optim.Adam(model.deephead.parameters())

In [14]:
wide_sch = torch.optim.lr_scheduler.StepLR(wide_opt, step_size=5)
deep_sch = torch.optim.lr_scheduler.MultiStepLR(deep_opt, milestones=[3, 8])
text_sch = torch.optim.lr_scheduler.StepLR(text_opt, step_size=5)
img_sch = torch.optim.lr_scheduler.MultiStepLR(deep_opt, milestones=[3, 8])
head_sch = torch.optim.lr_scheduler.StepLR(head_opt, step_size=5)

In [15]:
optimizers = {
    "wide": wide_opt,
    "deeptabular": deep_opt,
    "deeptext": text_opt,
    "deepimage": img_opt,
    "deephead": head_opt,
}
schedulers = {
    "wide": wide_sch,
    "deeptabular": deep_sch,
    "deeptext": text_sch,
    "deepimage": img_sch,
    "deephead": head_sch,
}

# Now...we have used pretrained word embeddings, so you do not want to
# initialise these  embeddings. However you might still want to initialise the
# other layers in the DeepText component. No probs, you can do that with the
# parameter pattern and your knowledge on regular  expressions. Here we are
# telling to the KaimingNormal initializer to NOT touch the  parameters whose
# name contains the string word_embed.
initializers = {
    "wide": KaimingNormal,
    "deeptabular": KaimingNormal,
    "deeptext": KaimingNormal(pattern=r"^(?!.*word_embed).*$"),
    "deepimage": KaimingNormal,
}

mean = [0.406, 0.456, 0.485]  # BGR
std = [0.225, 0.224, 0.229]  # BGR
transforms = [ToTensor, Normalize(mean=mean, std=std)]
callbacks = [
    LRHistory(n_epochs=10),
    EarlyStopping,
    ModelCheckpoint(filepath="model_weights/wd_out"),
]


In [16]:
trainer = Trainer(
    model,
    objective="rmse",
    initializers=initializers,
    optimizers=optimizers,
    lr_schedulers=schedulers,
    callbacks=callbacks,
    transforms=transforms,
)

/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/initializers.py:34: UserWarning: No initializer found for deephead
  warnings.warn(


In [17]:
trainer.fit(
    X_wide=X_wide,
    X_tab=X_tab,
    X_text=X_text,
    X_img=X_images,
    target=target,
    n_epochs=1000,
    batch_size=32,
    val_split=0.2,
)

  0%|                                                         | 0/500 [00:00<?, ?it/s]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid v

/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 2:  32%|████████▉                   | 160/500 [00:15<00:21, 16.13it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 2:  36%|██████████                  | 180/500 [00:16<00:19, 16.10it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 2:  40%|███████████                 | 198/500 [00:17<00:17, 17.51it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_d

/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 5:   0%|                            | 1/500 [00:08<1:08:21,  8.22s/it, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 5:  32%|████████▉                   | 159/500 [00:16<00:17, 19.65it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8

  0%|                                                         | 0/500 [00:00<?, ?it/s]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid v

epoch 9:  40%|███████████▏                | 199/500 [00:19<00:17, 17.26it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 9:  41%|███████████▌                | 207/500 [00:19<00:17, 16.80it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 9:  50%|██████████████              | 251/500 [00:22<00:13, 17.88it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
epoch 9:  51%|██████████████▎             | 256/500 [00:22<00:13, 18.31it/s, loss=nan]/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_wid

/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
/home/peilin/anaconda3/lib/python3.8/site-packages/pytorch_widedeep/training/_wd_dataset.py:187: RuntimeWarning: invalid value encountered in true_divide
  xdi = (xdi / xdi.max()).astype("float32")
valid: 100%|██████████████████████████████| 125/125 [00:11<00:00, 11.04it/s, loss=nan]


Model weights after training corresponds to the those of the final epoch which might not be the best performing weights. Usethe 'ModelCheckpoint' Callback to restore the best epoch weights.
